## --- Day 23: Amphipod ---


Amphipods can move up, down, left, or right so long as they are moving into an unoccupied open space. Each type of amphipod requires a different amount of energy to move one step: `Amber` amphipods require `1` energy per step, `Bronze` amphipods require `10` energy, `Copper` amphipods require `100`, and `Desert` ones require `1000`. The amphipods would like you to find a way to organize the amphipods that requires the least total energy.

However, because they are timid and stubborn, the amphipods have some extra rules:

- Amphipods will never stop on the space immediately outside any room. They can move into that space so long as they immediately continue moving. (Specifically, this refers to the four open spaces in the hallway that are directly above an amphipod starting position.)
- Amphipods will never move from the hallway into a room unless that room is their destination room and that room contains no amphipods which do not also have that room as their own destination. If an amphipod's starting room is not its destination room, it can stay in that room until it leaves the room. (For example, an Amber amphipod will not move from the hallway into the right three rooms, and will only move into the leftmost room if that room is empty or if it only contains other Amber amphipods.)
- Once an amphipod stops moving in the hallway, it will stay in that spot until it can move into a room. (That is, once any amphipod starts moving, any other amphipods currently in the hallway are locked in place and will not move again until they can move fully into a room.)

__What is the least energy required to organize the amphipods?__

### Example input
```
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
```

In [22]:
import json
from aoc_utils import PriorityQueue

class Burrow:
    # Room indices contain 0-2 amphipods, hallway spaces contain 0-1 amphipod
    rooms = [2, 4, 6, 8]
    spaces = [0, 1, 3, 5, 7, 9, 10]
    room_i = {"A": 2, "B": 4, "C": 6, "D": 8}
    weight = {"A": 1, "B": 10, "C": 100, "D": 1000}

    @classmethod
    def find_possible_moves(cls, state):
        next_states = []
        if type(state) == str:
            state = json.loads(state)
        def _try_move_to_room(orig_state, from_i):
            """
            If the amphipod at orig_state[from_i] can be moved to its room, create a next_state
            representing that move.
            """
            amphipod = orig_state[from_i][0]
            home_i = cls.room_i[amphipod]

            # can't go home if home is full, contains a diff't type of amphipod, or already there
            if len(orig_state[home_i]) == 2\
                or (len(orig_state[home_i]) == 1 and orig_state[home_i] != amphipod)\
                or home_i == from_i:
                return

            # can't go home if another amphipod is blocking a space on the way
            if home_i < from_i:
                path = range(home_i, from_i)        # home is to the left
            else:
                path = range(from_i+1, home_i)      # home is to the right
            if any(i not in cls.rooms and len(orig_state[i]) > 0 for i in path):
                return

            next_state = orig_state.copy()
            next_state[home_i] = amphipod + next_state[home_i]
            next_state[from_i] = next_state[from_i][1:]
            next_cost = cls.calculate_move_cost(orig_state, from_i, home_i)
            next_est = next_cost + cls.estimate(next_state)
            next_states.append((next_state, next_est, next_cost))

        def _try_move_to_spaces(orig_state, from_i, i_range):
            """
            Check all values in i_range for spaces (not rooms) that amphipod at room from_i
            can potentially move to.
            When possible, make a copy of the state, do the swap, append the copy to next_moves
            """
            # Don't try to move "solved" amphipods
            if from_i == cls.room_i[orig_state[from_i][0]]:
                if len(orig_state[from_i]) == 1:
                    # This amphipod is solved and the only one in its room
                    return
                elif len(orig_state[from_i]) == 2\
                    and orig_state[from_i][0] == orig_state[from_i][1]:
                    # Both amphipods are in their correct room
                    return

            for to_i in i_range:
                if to_i in cls.spaces:
                    if len(orig_state[to_i]) > 0:
                        # to_i is an occupied space, can't continue any farther
                        return
                    else:
                        # to_i is an available space, make the move
                        next_state = orig_state.copy()
                        next_state[to_i] = next_state[from_i][0]
                        next_state[from_i] = next_state[from_i][1:]
                        next_cost = cls.calculate_move_cost(orig_state, from_i, to_i)
                        next_est = next_cost + cls.estimate(next_state)
                        next_states.append((next_state, next_est, next_cost))
           
        # Find movable amphipods and try to move to all open destinations
        for i in range(len(state)):
            if len(state[i]) > 0:
                # Any amphipod can potentially move to its room
                _try_move_to_room(state, i)

                if i in cls.rooms:
                    # From inside a room, amphipod can move to any available space or to its room
                    _try_move_to_spaces(state, i, range(i-1, -1, -1))
                    _try_move_to_spaces(state, i, range(i+1, 11))

        return next_states

    @classmethod
    def calculate_move_cost(cls, state, from_i, to_i):
        """Calculates cost of moving amphipod"""
        # Cost of moving RIGHT/LEFT
        cost = abs(from_i - to_i)

        # If moving from a room, cost UP to exit room
        if from_i in cls.rooms:
            # when room len 2, top amphipod exit cost 1; len 1, bottom amphipod exit cost 2
            cost += 3 - len(state[from_i])

        # If moving to a room, cost DOWN to enter room
        if to_i in cls.rooms:
            # when len 1, cost 1 into top space; len 0 cost 2 into bottom
            cost += 2 - len(state[to_i])

        # Multiply by the weight of the amphipod moving
        amphipod = state[from_i][0]
        cost *= cls.weight[amphipod]

        return cost

    @classmethod
    def estimate(cls, state):
        """Estimates the total energy of the state from the solution"""
        estimate = 0

        for i in range(len(state)):
            if len(state[i]) > 0:
                amphipod = state[i][0]
                home_i = cls.room_i[amphipod]
                estimate += abs(home_i - i) * cls.weight[amphipod]
            if len(state[i]) == 2:
                amphipod = state[i][1]
                home_i = cls.room_i[amphipod]
                estimate += abs(home_i - i) * cls.weight[amphipod]

        return estimate

    @classmethod
    def solved(cls, state):
        if type(state) == str:
            state = json.loads(state)
        return state == ["","", "AA", "", "BB", "", "CC", "", "DD", "", ""]

    @classmethod
    def organize(cls, initial_state):
        pq = PriorityQueue()
        current_state = initial_state
        current_cost = 0

        while not cls.solved(current_state):
            next_states = cls.find_possible_moves(current_state)
            for state, est, cost in next_states:
                pq.update(json.dumps(state), (est, current_cost + cost))
            current_state, (est, current_cost) = pq.pop()

        return current_cost


    @classmethod
    def print_state(cls, state):
        hallway_str = "#"
        for i in range(len(state)):
            if i in cls.rooms or state[i] == "":
                hallway_str += "."
            else:
                hallway_str += state[i]

        upper_str = "###" + "#".join(state[room].rjust(2,".")[0] for room in cls.rooms) + "###"
        lower_str = "  #" + "#".join(state[room].rjust(2,".")[1] for room in cls.rooms) + "#"
        print("#" * 13)
        print(hallway_str + "#")
        print(upper_str)
        print(lower_str)
        print("  " + "#"*9)

In [27]:
# Example
ex1_input = ["", "", "BA", "", "CD", "", "BC", "", "DA", "", ""]
assert 12521 == Burrow.organize(ex1_input)


In [28]:
# Part 1 solution
p1_input = ["", "", "BD", "", "CD", "", "CA", "", "BA", "", ""]
Burrow.organize(p1_input)

18051